In [38]:
import subprocess
import psycopg2
import re

In [39]:


# Connect to an existing database
executable_path = './main-pol-sta'
p = "3" # A prime 
with psycopg2.connect("dbname=p-rank-2") as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Query the database and obtain data as Python objects.
        cur.execute("""
                    WITH OrderedRecords AS (
                        SELECT
                            D,
                            L_pol,
                            ROW_NUMBER() OVER (ORDER BY D) AS RowNum
                        FROM p_3
                    )
                    SELECT D, L_pol
                    FROM OrderedRecords
                    WHERE RowNum >= (SELECT RowNum FROM OrderedRecords WHERE D = 134059)
                    ORDER BY D
                    LIMIT 1;
                    """)
        records = cur.fetchall()
        
        for record in records:
            print(record[0])
            print(record[1][1:-1])
            pol_vec = record[1][1:-1].split(', ')
            print(pol_vec)
            relations = []
            for pol in pol_vec:
                program_arguments = [p, pol]
                # Construct the command by combining the executable path and arguments
                command = [executable_path]+program_arguments
                
                # Run the C program with arguments and capture its output
                result = subprocess.run(command, capture_output=True, text=True)
                full_output = result.stdout
                marker_position = full_output.find('"""')
                if marker_position != -1:
                    output = full_output[marker_position+3:]
                    modified_s = re.sub(r'(\d)(?![,\]])', r'\1*', output)
                    relations.append(modified_s)
                    print(output, "\n")
                else:
                    print("The C program crashed\n")
                print("Done: ", relations)

        # Make the changes to the database persistent
        conn.commit()

134059
s^6 - 48*s^4 + 576*s^2 + 134059, s^6 + 172*s^4 + 7396*s^2 + 134059, s^6 - 156*s^4 + 6084*s^2 + 134059, s^6 - 104*s^4 + 2704*s^2 + 134059
['s^6 - 48*s^4 + 576*s^2 + 134059', 's^6 + 172*s^4 + 7396*s^2 + 134059', 's^6 - 156*s^4 + 6084*s^2 + 134059', 's^6 - 104*s^4 + 2704*s^2 + 134059']
[a_c^1, a^6a_b^1b_c^1, a^3a_b^2c^6]  

Done:  ['[a_c^1, a^6*a_b^1*b_c^1, a^3*a_b^2*c^6] ']
[a_b^2a_c^1b^3, b_c^1, a_b^1a_c^1c^3]  

Done:  ['[a_c^1, a^6*a_b^1*b_c^1, a^3*a_b^2*c^6] ', '[a_b^2*a_c^1*b^3, b_c^1, a_b^1*a_c^1*c^3] ']
[a_b^1, a^3a_c^2b^3, a^3a_b^2a_c^2b_c^2]  

Done:  ['[a_c^1, a^6*a_b^1*b_c^1, a^3*a_b^2*c^6] ', '[a_b^2*a_c^1*b^3, b_c^1, a_b^1*a_c^1*c^3] ', '[a_b^1, a^3*a_c^2*b^3, a^3*a_b^2*a_c^2*b_c^2] ']
The C program crashed

Done:  ['[a_c^1, a^6*a_b^1*b_c^1, a^3*a_b^2*c^6] ', '[a_b^2*a_c^1*b^3, b_c^1, a_b^1*a_c^1*c^3] ', '[a_b^1, a^3*a_c^2*b^3, a^3*a_b^2*a_c^2*b_c^2] ']
